/bin/sh: 1: pip: not found


In [60]:
import pandas as pd 
import numpy as np
import types

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import cross_validate

import os

In [4]:
# train_data = pd.read_csv('../input/train.csv')

train_data = pd.read_csv('train.csv')
y_train = train_data['Survived'].values
#train_data = train_data.drop(columns="Survived")

test_data = pd.read_csv('test.csv')

full_data = pd.concat([train_data, test_data], sort=False)

train_data.sample(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
325,326,1,1,"Young, Miss. Marie Grice",female,36.0,0,0,PC 17760,135.6333,C32,C
835,836,1,1,"Compton, Miss. Sara Rebecca",female,39.0,1,1,PC 17756,83.1583,E49,C
294,295,0,3,"Mineff, Mr. Ivan",male,24.0,0,0,349233,7.8958,NaN,S
371,372,0,3,"Wiklund, Mr. Jakob Alfred",male,18.0,1,0,3101267,6.4958,NaN,S
413,414,0,2,"Cunningham, Mr. Alfred Fleming",male,NaN,0,0,239853,0.0000,NaN,S
819,820,0,3,"Skoog, Master. Karl Thorsten",male,10.0,3,2,347088,27.9000,NaN,S
495,496,0,3,"Yousseff, Mr. Gerious",male,NaN,0,0,2627,14.4583,NaN,C
558,559,1,1,"Taussig, Mrs. Emil (Tillie Mandelbaum)",female,39.0,1,1,110413,79.6500,E67,S
92,93,0,1,"Chaffee, Mr. Herbert Fuller",male,46.0,1,0,W.E.P. 5734,61.1750,E31,S
840,841,0,3,"Alhomaki, Mr. Ilmari Rudolf",male,20.0,0,0,SOTON/O2 3101287,7.9250,NaN,S


In [70]:
def get_feats(df):
    feats = df.select_dtypes(include=[np.int64]).columns.values
    black_list = ['PassengerId', 'Survived']
    
    return [feat for feat in feats if feat not in black_list]

In [6]:
def isnan_desc(column, dataset):
    assert isinstance(column, str), "column should be a string"
    
    if dataset[column].isna().any() == True:
        isnan = True
    else : 
        isnan = False
    print("Has a NaN: {} \ndescribe:\n{}".format(isnan,dataset[column].describe()))

In [33]:
full_data['Name'].map(lambda x: x.split(',')[1].split('.')[0].lower().strip() ).value_counts()

mr              757
miss            260
mrs             197
master           61
rev               8
dr                8
col               4
ms                2
mlle              2
major             2
lady              1
jonkheer          1
sir               1
dona              1
capt              1
the countess      1
don               1
mme               1
Name: Name, dtype: int64

In [7]:
isnan_desc("Embarked",full_data)

Has a NaN: True 
describe:
count     1307
unique       3
top          S
freq       914
Name: Embarked, dtype: object


In [67]:
def feature_enginnering(dataset):
    dataset['Cabin'] = dataset['Cabin'].fillna('Missing')
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
    
    dataset['title'] = dataset['Name'].map(lambda x: x.split(',')[1].split('.')[0].lower().strip() )
    popular_titles = ["mr", "miss", "mrs", "master", "dr", "rev"]
    dataset['title_norm'] = dataset['title'].map(lambda x: x if x in popular_titles else "other")
    dataset['title_norm_cat'] = pd.factorize( dataset['title_norm'] )[0]
    
    dataset["Sex"] = pd.factorize(dataset["Sex"])[0]
    dataset["Embarked"] = pd.factorize(dataset["Embarked"])[0]
    dataset["Cabin"] = pd.factorize(dataset["Cabin"])[0]
    dataset["Ticket"] = pd.factorize(dataset["Ticket"])[0]
    
    dataset.drop('Name', axis=1)
    dataset.drop('PassengerId',axis=1)
    dataset.drop('title', axis=1)
    dataset.drop('title_norm', axis=1)

    return dataset

In [82]:
def make_prediction(df_train, df_test, model, output_file_name):
    train = feature_enginnering(train_data)
    test = feature_enginnering(test_data)
    
    feats = get_feats(train)
    
    X_train = train[feats].values
    y_train = train['Survived'].values
    X_test = test[feats]

    print(model)
    model.fit(X_train, y_train)

    test['Survived'] = model.predict(X_test)
    test[ ['PassengerId', 'Survived'] ].to_csv('../titanic/output/' + output_file_name, index=False)

In [83]:
make_prediction(train_data, test_data, LogisticRegression(), 'linear.csv')

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)


/home/adam/DEV/python/kaggle/venvkaggle/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
